In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator,img_to_array,array_to_img,load_img
from keras import backend as K

In [3]:
model=Sequential()

In [4]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 100
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

### First Layer

In [5]:
model.add(Conv2D(32,(3,3),input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

### Second Layer 

In [6]:
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

### Third Layer

In [7]:
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# the model so far outputs 3D feature maps (height, width, features)

In [8]:
model.add(Flatten())# this converts our 3D feature maps to 1D feature vectors

In [9]:
model.add(Dense(64))
model.add(Activation('relu'))

In [10]:
model.add(Dropout(0.5))

In [11]:
model.add(Dense(1))
model.add(Activation('sigmoid'))

### Compile Model

In [12]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

### Prepare Dataset

In [13]:
# this is the augmentation configuration we will use for training
train_datagen=ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [14]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen=ImageDataGenerator(rescale=1./255)

In [15]:
train_generator=train_datagen.flow_from_directory('preview/train',
                                                  target_size=(150,150),
                                                  batch_size=batch_size,
                                                  class_mode='binary')

Found 2000 images belonging to 2 classes.


In [16]:
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'preview/validation',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

Found 800 images belonging to 2 classes.


### Train Model

In [17]:
#Model is trained
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

#model.save_weights('first_try.h5') 
model.save('64x3-CNN.model') 

Epoch 1/100
125/125 [==============================] - 28s 223ms/step - loss: 0.6931 - accuracy: 0.5400 - val_loss: 0.6283 - val_accuracy: 0.6438
Epoch 2/100
125/125 [==============================] - 29s 235ms/step - loss: 0.6543 - accuracy: 0.6240 - val_loss: 0.5970 - val_accuracy: 0.6687
Epoch 3/100
125/125 [==============================] - 29s 233ms/step - loss: 0.6282 - accuracy: 0.6525 - val_loss: 0.6749 - val_accuracy: 0.6662
Epoch 4/100
125/125 [==============================] - 29s 233ms/step - loss: 0.6139 - accuracy: 0.6730 - val_loss: 0.5010 - val_accuracy: 0.6538
Epoch 5/100
125/125 [==============================] - 29s 232ms/step - loss: 0.6000 - accuracy: 0.6815 - val_loss: 0.5796 - val_accuracy: 0.6963
Epoch 6/100
125/125 [==============================] - 29s 232ms/step - loss: 0.5685 - accuracy: 0.7100 - val_loss: 0.7005 - val_accuracy: 0.5825
Epoch 7/100
125/125 [==============================] - 29s 232ms/step - loss: 0.5655 - accuracy: 0.7105 - val_loss: 0.6232 -

### Test Model

In [18]:
import numpy as np
from keras.preprocessing import image

In [30]:
test_image=image.load_img('preview/cat.1014.jpg',target_size=(150,150))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis = 0)

result=model.predict(test_image)

train_generator.class_indices

if result[0][0] >=0.5:
    prediction='Dog'
else:
    prediction='Cat'
print(prediction)

Cat
